In [1]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.data.target import apply_selection_mask_to_models

from gammapy.modeling.models import Models, PowerLawSpectralModel

import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
import logging

In [2]:
from asgardpy.data.dataset_3d import Dataset3DGeneration

In [3]:
log = logging.getLogger("test 3d dataset")

In [32]:
#config_file = "/path/to/Config.yaml"
config_file = "/fefs/aswg/workspace/chaitanya.priyadarshi/analyze_pg1553_fermipy.yaml"

In [33]:
config = AsgardpyConfig()

In [34]:
%%time
config_main = config.read(config_file)

CPU times: user 34.1 ms, sys: 2.88 ms, total: 37 ms
Wall time: 34 ms


In [35]:
for g in config_main.general:
    print(g)

('log', LogConfig(level='info', filename='', filemode='w', format='', datefmt=''))
('outdir', PosixPath('/fefs/aswg/workspace/chaitanya.priyadarshi/real_data/AGN/PG1553/v09x/tailcut84/DL3/interp_irf_en_dep/no_tuning/geff_80/teff_80/20220511_dec931/joint_fit/fermipy/test2'))
('n_jobs', 1)
('steps', [<AnalysisStepEnum.datasets_3d: 'datasets-3d'>, <AnalysisStepEnum.datasets_1d: 'datasets-1d'>, <AnalysisStepEnum.fit: 'fit'>, <AnalysisStepEnum.flux_points: 'flux-points'>])
('overwrite', True)
('stacked_dataset', True)


# Steps mentioned in the main config file

In [36]:
config_main.general.steps

[<AnalysisStepEnum.datasets_3d: 'datasets-3d'>,
 <AnalysisStepEnum.datasets_1d: 'datasets-1d'>,
 <AnalysisStepEnum.fit: 'fit'>,
 <AnalysisStepEnum.flux_points: 'flux-points'>]

# Target source information

In [37]:
for c in config_main.target:
    print(c)

('source_name', '4FGL J1555.7+1111')
('sky_position', SkyCoordConfig(frame=<FrameEnum.icrs: 'icrs'>, lon=<Angle 238.92934976 deg>, lat=<Angle 11.19010155 deg>))
('use_uniform_position', True)
('models_file', PosixPath('/fefs/aswg/workspace/chaitanya.priyadarshi/pg1553_model_ecpl_fermipy.yaml'))
('extended', False)
('components', [SkyModelComponent(name='', type='SkyModel', spectral=SpectralModelConfig(type='', parameters=[ModelParams(name='', value=1.0, unit=' ', error=0.1, min=0.1, max=10.0, frozen=True)], ebl_abs=EBLAbsorptionModel(filename=PosixPath('.'), reference='dominguez', type='EBLAbsorptionNormSpectralModel', redshift=0.4, alpha_norm=1.0)), spatial=SpatialModelConfig(type='', parameters=[ModelParams(name='', value=1.0, unit=' ', error=0.1, min=0.1, max=10.0, frozen=True)]))])
('covariance', 'None')
('from_3d', False)
('roi_selection', RoISelectionConfig(roi_radius=<Angle 0. deg>, free_sources=['None']))


# 3D Datasets informations

In [38]:
for cc in config_main.dataset3d:
    print(cc)

('type', <ReductionTypeEnum.cube: '3d'>)
('instruments', [Dataset3DBaseConfig(name='Fermi-LAT', io=[InputConfig(type='lat', input_dir=PosixPath('/fefs/aswg/workspace/lst1-agn-paper/LAT/PG1553+113/Analysis3/LST_times/base'), glob_pattern={'events': 'ft1*.fits*', 'edisp': 'drm*.fits*', 'xml_model': 'fit1_0*.xml', 'exposure': 'bexpmap_0*.fits*', 'psf': 'psf*.fits*'}), InputConfig(type='lat-aux', input_dir=PosixPath('/fefs/aswg/workspace/lst1-agn-paper/LAT/diffuse_models'), glob_pattern={'gal_diffuse': '', 'iso_diffuse': ''})], dataset_info=Dataset3DInfoConfig(name='Fermi-LAT', key=['00', '01'], map_selection=[<MapSelectionEnum.counts: 'counts'>, <MapSelectionEnum.exposure: 'exposure'>, <MapSelectionEnum.background: 'background'>, <MapSelectionEnum.psf: 'psf'>, <MapSelectionEnum.edisp: 'edisp'>], geom=GeomConfig(wcs=WcsConfig(skydir=SkyCoordConfig(frame=<FrameEnum.icrs: 'icrs'>, lon=<Angle 0. deg>, lat=<Angle 0. deg>), binsize=<Angle 0.1 deg>, proj=<ProjectionEnum.tan: 'TAN'>, map_frame_sh

In [39]:
for cc in config_main.fit_params:
    print(cc)

('fit_range', EnergyRangeConfig(min=<Quantity 300. MeV>, max=<Quantity 1. TeV>))
('backend', <BackendEnum.minuit: 'minuit'>)
('optimize_opts', {})
('covariance_opts', {})
('confidence_opts', {})
('store_trace', True)


In [40]:
config_main.general.log.dict()

{'level': 'info', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}

In [41]:
%%time
analysis = AsgardpyAnalysis(config_main)

Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}


CPU times: user 31.3 ms, sys: 3.12 ms, total: 34.4 ms
Wall time: 31.4 ms


In [42]:
analysis

# Prepare for a single 3D Dataset, based on instrument and key_name

In [43]:
instruments_list = config_main.dataset3d.instruments
print(len(instruments_list), "number of 3D dataset information provided")

config_3d_dataset = instruments_list[0]
print(f"Instrument selected is {config_3d_dataset.name}")

1 number of 3D dataset information provided
Instrument selected is Fermi-LAT


In [44]:
key_names = config_3d_dataset.dataset_info.key
print(f"The list of different keys or modes of observations for the selected instrument are {key_names}")

The list of different keys or modes of observations for the selected instrument are ['00', '01']


# Run each Dataset3DGeneration step

In [45]:
%%time
generate_3d_dataset = Dataset3DGeneration(
    log, config_3d_dataset, config_main
)

CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 40.5 µs


## Initialized variables

In [46]:
print(generate_3d_dataset.config_target)

source_name='4FGL J1555.7+1111' sky_position=SkyCoordConfig(frame=<FrameEnum.icrs: 'icrs'>, lon=<Angle 238.92934976 deg>, lat=<Angle 11.19010155 deg>) use_uniform_position=True models_file=PosixPath('/fefs/aswg/workspace/chaitanya.priyadarshi/pg1553_model_ecpl_fermipy.yaml') extended=False components=[SkyModelComponent(name='', type='SkyModel', spectral=SpectralModelConfig(type='', parameters=[ModelParams(name='', value=1.0, unit=' ', error=0.1, min=0.1, max=10.0, frozen=True)], ebl_abs=EBLAbsorptionModel(filename=PosixPath('.'), reference='dominguez', type='EBLAbsorptionNormSpectralModel', redshift=0.4, alpha_norm=1.0)), spatial=SpatialModelConfig(type='', parameters=[ModelParams(name='', value=1.0, unit=' ', error=0.1, min=0.1, max=10.0, frozen=True)]))] covariance='None' from_3d=False roi_selection=RoISelectionConfig(roi_radius=<Angle 0. deg>, free_sources=['None'])


In [47]:
print(generate_3d_dataset.exclusion_mask)

None


In [48]:
print(generate_3d_dataset.irfs)

{'exposure': None, 'psf': None, 'edisp': None, 'edisp_kernel': None, 'edisp_interp_kernel': None, 'exposure_interp': None}


In [49]:
print(generate_3d_dataset.events)

{'events': None, 'event_fits': None, 'gti': None, 'counts_map': None}


In [50]:
print(generate_3d_dataset.diffuse_models)

{'gal_diffuse': None, 'iso_diffuse': None, 'gal_diffuse_cutout': None}


In [51]:
print(generate_3d_dataset.list_sources)

[]


### First check for the given file list if they are readable or not.

In [52]:
%%time
file_list = generate_3d_dataset.read_to_objects(key_names[0])

Selecting the file with name containing 00
Selecting the file with name containing 00
Selecting the file with name containing 00
Selecting the file with name containing 00
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set MJD-OBS to 59311.500100 from DATE-OBS.
Set MJD-END to 59639.499613 from DATE-END'. [astropy.wcs.wcs]
Set MJD-OBS to 59311.500100 from DATE-OBS.
Set MJD-END to 59639.499613 from DATE-END'.


KeyError: "No object found with tag: ''"

In [31]:
print(file_list)

NameError: name 'file_list' is not defined

In [ ]:
for k in file_list.keys():
    print(k, file_list[k])

In [ ]:
print(generate_3d_dataset.irfs["exposure"])

In [ ]:
print(generate_3d_dataset.irfs["psf"])

In [ ]:
print(generate_3d_dataset.irfs["edisp"].info())

In [ ]:
print(generate_3d_dataset.diffuse_models["gal_diffuse"])

In [ ]:
print(generate_3d_dataset.diffuse_models["iso_diffuse"])

In [ ]:
print(len(generate_3d_dataset.list_sources), "number of sources read from XML file")

In [ ]:
for src in generate_3d_dataset.list_sources:
    print(src)

In [ ]:
print("Number of free parameters in the whole list of SkyModels:", len(Models(generate_3d_dataset.list_sources).parameters.free_parameters))

In [ ]:
print("Names of the sources in the whole list of SkyModels:", Models(generate_3d_dataset.list_sources).names)

### Start preparing objects to create the counts map

In [ ]:
%%time
generate_3d_dataset.set_energy_dispersion_matrix()

In [ ]:
print(generate_3d_dataset.irfs["edisp_kernel"])
generate_3d_dataset.irfs["edisp_kernel"].peek()

In [ ]:
%%time
generate_3d_dataset.load_events(file_list["events_file"])

### EventList and GTI

In [ ]:
generate_3d_dataset.events["events"].peek()

In [ ]:
print(generate_3d_dataset.events["gti"])

In [ ]:
%%time
# Source position from the Events file
source_pos, evts_radius = generate_3d_dataset.get_source_skycoord()
print(source_pos, evts_radius)

## Create the Counts Map

In [ ]:
%%time
generate_3d_dataset.create_counts_map(source_pos, evts_radius)

In [ ]:
generate_3d_dataset.events["counts_map"]

In [ ]:
generate_3d_dataset.events["counts_map"].plot_interactive()

In [ ]:
generate_3d_dataset.events["counts_map"].sum_over_axes().smooth(2).plot(stretch="sqrt", add_cbar=True)

### Exclusion mask

In [ ]:
%%time
generate_3d_dataset.create_exclusion_mask()

In [ ]:
print(generate_3d_dataset.exclusion_mask)

In [ ]:
generate_3d_dataset.exclusion_mask.plot_interactive()

### IRF interpolators

In [ ]:
%%time
generate_3d_dataset.set_edisp_interpolator()

In [ ]:
print(generate_3d_dataset.irfs["edisp_interp_kernel"])
generate_3d_dataset.irfs["edisp_interp_kernel"].peek()

In [ ]:
%%time
generate_3d_dataset.set_exposure_interpolator()

In [ ]:
print(generate_3d_dataset.irfs["exposure_interp"])

### Diffuse background cutout

In [ ]:
%%time
generate_3d_dataset.generate_diffuse_background_cutout()

In [ ]:
print(Models(generate_3d_dataset.list_sources)["diffuse-iem"])

In [ ]:
print(generate_3d_dataset.diffuse_models["gal_diffuse_cutout"])

# Apply Exclusion mask to models

In [ ]:
generate_3d_dataset.list_sources = apply_selection_mask_to_models(
    generate_3d_dataset.list_sources,
    target_source=generate_3d_dataset.config_target.source_name,
    selection_mask=generate_3d_dataset.exclusion_mask,
)

In [ ]:
for m in generate_3d_dataset.list_sources:
    print(m)

### Produce MapDataset

In [ ]:
%%time
dataset = generate_3d_dataset.generate_dataset(key_names[0])

In [ ]:
print(dataset)

In [ ]:
for m in generate_3d_dataset.list_sources:
    print(m)

# Assigning models to the dataset

In [ ]:
%%time
dataset.models = generate_3d_dataset.list_sources

In [ ]:
%%time
print(dataset)

# Some basic plots from the Dataset

In [ ]:
dataset.mask_safe.plot_interactive()

In [ ]:
fig = plt.figure(figsize=(6, 3.8), dpi=120)
vmin, vmax = np.percentile(
    dataset.counts.sum_over_axes().smooth("0.1 deg").data, 
    [5, 99.9]
)
ax = dataset.counts.sum_over_axes().smooth("0.2 deg").plot(
    vmin=vmin, vmax=vmax, add_cbar=True, stretch="sqrt",
)
ax.set_title('CountsMap')

In [ ]:
dataset.plot_residuals_spectral()

In [ ]:
dataset.plot_residuals_spatial()

In [ ]:
dataset.info_dict()

In [ ]:
plt.figure(figsize=(8, 5))
axs = plt.subplot(111, projection=dataset.counts.geom.wcs)

dataset.counts.sum_over_axes().smooth(0.05 * u.deg).plot(
    ax=axs, stretch="sqrt", add_cbar=True, cmap="afmhot"
)
dataset.models.plot_regions(ax=axs, color="white")
axs.set_title(dataset.name)

plt.show()

In [ ]:
dataset.exposure.plot_interactive(add_cbar=True)

In [ ]:
dataset.psf.plot_containment_radius_vs_energy()

In [ ]:
plt.figure(figsize=(8, 5))

energy = [100, 300, 1000] * u.GeV
dataset.psf.plot_psf_vs_rad(energy_true=energy)

spectrum = PowerLawSpectralModel(index=1.25)
psf_mean = dataset.psf.to_image(spectrum=spectrum)
psf_mean.plot_psf_vs_rad(c="k", ls="--", energy_true=[500] * u.GeV)

plt.xlim(1e-3, 0.3)
plt.ylim(1e3, 1e6)
plt.legend()

In [ ]:
radius = dataset.psf.containment_radius(energy_true=1 * u.TeV, fraction=0.95)
print(radius)

In [ ]:
plt.figure()
edisp_kernel = dataset.edisp.get_edisp_kernel()
edisp_kernel.plot_matrix()

In [ ]:
plt.figure()
npred = dataset.npred()
npred.sum_over_axes().plot(add_cbar=True)